In [1]:
import pandas as pd
import lightgbm as lgb
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

print("--- Entraînement du Modèle de Prédiction de Taux ---")

# --- 1. Chargement des données et du modèle de risque expert ---
input_path = "../../data/processed/dataset_clean_no_outliers.parquet"
model_expert_path = "../../models/model_risk_of_default.pkl"
model_expert_cols_path = "../../models/model_1_columns.pkl"

df = pd.read_parquet(input_path)
model_expert = joblib.load(model_expert_path)
model_expert_columns = joblib.load(model_expert_cols_path)
print("✅ Données et modèle expert chargés.")

# --- 2. Calcul des scores de risque pour toute la base ---
# Préparation des données complètes pour l'expert
features_to_exclude = [
    'loan_status', 'is_default', 'id', 'grade', 'sub_grade', 'int_rate', 'installment', 'last_pymnt_d',
    'emp_title', 'emp_title_clean', 'title', 'zip_code', 'job_category', 'csp_category', 
    'issue_d', 'earliest_cr_line'
]
X_expert = df.drop(columns=features_to_exclude, errors='ignore')
X_expert = pd.get_dummies(X_expert, drop_first=True, dtype=float)
X_expert = X_expert.reindex(columns=model_expert_columns, fill_value=0)

df['risk_score'] = model_expert.predict_proba(X_expert)[:, 1]
print("✅ Scores de risque calculés pour toute la base.")

# --- 3. Préparation des données pour le modèle de tarification ---
# La cible (Y) est le taux d'intérêt
y_target = df['int_rate'] * 100 # On prédit le taux en % (ex: 7.5)

# Les features (X) sont le score de risque et les infos du prêt
features_for_rate_model = [
    'risk_score',
    'loan_amnt',
    'annual_inc',
    'dti',
    'emp_length',
    'home_ownership',
    'term'
]
X_rate = df[features_for_rate_model]
X_rate = pd.get_dummies(X_rate, drop_first=True, dtype=float)

# --- 4. Entraînement du modèle de régression ---
X_train, X_test, y_train, y_test = train_test_split(X_rate, y_target, test_size=0.2, random_state=42)

rate_model = lgb.LGBMRegressor(random_state=42)
print("Entraînement du modèle de tarification...")
rate_model.fit(X_train, y_train)
print("✅ Modèle de tarification entraîné.")

# --- 5. Évaluation du modèle ---
y_pred = rate_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"\n--- Performance du Modèle de Tarification ---")
print(f"Score R² : {r2:.4f}")
print(f"Erreur Absolue Moyenne (MAE) : {mae:.4f} points de pourcentage")

# --- 6. Sauvegarde ---
rate_model_path = "../../models/rate_prediction_model.pkl"
rate_model_cols_path = "../../models/rate_model_columns.pkl"

joblib.dump(rate_model, rate_model_path)
joblib.dump(X_rate.columns.tolist(), rate_model_cols_path)
print(f"\n✅ Modèle de tarification et ses colonnes sauvegardés.")

--- Entraînement du Modèle de Prédiction de Taux ---
✅ Données et modèle expert chargés.
✅ Scores de risque calculés pour toute la base.
Entraînement du modèle de tarification...
✅ Modèle de tarification entraîné.

--- Performance du Modèle de Tarification ---
Score R² : 0.3613
Erreur Absolue Moyenne (MAE) : 307.7839 points de pourcentage

✅ Modèle de tarification et ses colonnes sauvegardés.
